# Calculate spend based biodiversity impact factors PDF/€
This notebook demonstrates how to calculate spend based biodiversity impact factors by combining data from lc-impact and exiobase

In [1]:
# import required libraries
import pymrio
import numpy as np
import pandas as pd
import pycountry as pyc

In [ ]:
# exiobase 2011 is used for calculating share of stressor for each region-product pair
exio3_11 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2011_pxp.zip")
# exiobase 2019 is used for impact factors
exio3_19 = pymrio.parse_exiobase3(path="/Users/tterimaa/code/projects/impact-factors/IOT_2019_pxp")

In [30]:
# lc-impact data for water consumption
lci_water = pd.read_excel("/Users/tterimaa/code/projects/impact-factors/lc-impact/LC-Impact/12-water consumption/CFs_water_consumption_ecosystems_20180831.xlsx", sheet_name="CF per countries", dtype={1: float, 2: float}, skiprows=2)

In [3]:
# aggregate all blue water consumption related drivers
groups = exio3_11.satellite.get_index(as_dict=True, grouping_pattern={"Water Consumption Blue.*": "Water Consumption Blue -– Total"})

exio3_11.satellite_agg = exio3_11.satellite.copy(new_name="Aggregated blue water consumption accounts")

for df_name, df in zip(exio3_11.satellite_agg.get_DataFrame(data=False, with_unit=True, with_population=False),
exio3_11.satellite_agg.get_DataFrame(data=True, with_unit=True, with_population=False)):
    if df_name == "unit":
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).apply(lambda x: " & ".join(x.unit.unique()))
    else:
        exio3_11.satellite_agg.__dict__[df_name] = df.groupby(groups).sum()

In [4]:
# diagonalize total blue water consumption and calculate blue water consumption accounts
et1_diag = exio3_11.satellite_agg.diag_stressor(("Water Consumption Blue -– Total"))
exio3_11.et1_diag = et1_diag
exio3_11.calc_all()

/nix/store/byn37i6kwq65q36ipaajfqyl8lwp749v-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/byn37i6kwq65q36ipaajfqyl8lwp749v-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/byn37i6kwq65q36ipaajfqyl8lwp749v-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/nix/store/byn37i6kwq65q36ipaajfqyl8lwp749v-python3.12-pymrio/lib/python3.12/site-packages/pymrio/core/mriosystem.p

In [86]:
# calculate dr_s - share of the driver of biodiversity loss in impact region i from the total amount of the driver that is driven by consumption in consumption region j, product sector k
dr_s = pd.DataFrame()
for series_name, series in exio3_11.et1_diag.D_cba.items():
    series_sum = series.sum()
    dr_s[series_name] = series / series_sum

/private/tmp/nix-shell-8849-0/nix-shell-96303-0/ipykernel_58926/2643969149.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dr_s[series_name] = series / series_sum
/private/tmp/nix-shell-8849-0/nix-shell-96303-0/ipykernel_58926/2643969149.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dr_s[series_name] = series / series_sum
/private/tmp/nix-shell-8849-0/nix-shell-96303-0/ipykernel_58926/2643969149.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, w

In [87]:
# define rest of the world regions and calculate R 

# for this harmonization we need to know which countries go into:
# 1. rest of the world europe
# 2. rest of the world asia pacific
# 3. rest of the world middle east
# 4. rest of the world africa
# 5. rest of the world americas

row_eu_countries = {
    'AL': 'Albania',
    'AZ': 'Azerbaijan',
    'BA': 'Bosnia and Herzegovina',
    'BY': 'Belarus',
    'FO': 'Faroe Islands',
    'IS': 'Iceland',
    'GL': 'Greenland',
    'GE': 'Georgia',
    'MK': 'Macedonia',
    'MD': 'Moldova',
    'ME': 'Montenegro',
    'RS': 'Serbia',
    'SJ': 'Svalbard',
    'UA': 'Ukraine',
    # overseas territories of EU countries 
    'GP': 'Guadeloupe',
    'PF': 'French Polynesia',
    'FK': 'Falkland Islands (Malvinas)',
    'GF': 'French Guiana',
    'VG': 'British Virgin Islands',
    'RE': 'Réunion',
    'VC': 'Saint Vincent and the Grenadines',
}

row_asia_pacific_countries = {
    'BD': 'Bangladesh', 'BN': 'Brunei', 'BT': 'Bhutan', 'CK': 'Cook Islands', 'KH': 'Cambodia', 'FJ': 'Fiji',
    'KZ': 'Kazakhstan', 'KI': 'Kiribati', 'KG': 'Kyrgyzstan', 'LA': 'Laos', 'MY': 'Malaysia',
    'MN': 'Mongolia', 'MM': 'Myanmar (Burma)', 'NP': 'Nepal', 'NC': 'New Caledonia', 'NZ': 'New Zealand',
    'KP': 'North Korea', 'PK': 'Pakistan', 'PG': 'Papua New Guinea', 'PH': 'Philippines',
    'WS': 'Samoa', 'SB': 'Solomon Islands', 'LK': 'Sri Lanka',
    'TJ': 'Tajikistan', 'TH': 'Thailand', 'TM': 'Turkmenistan',
    'UZ': 'Uzbekistan', 'VU': 'Vanuatu', 'VN': 'Vietnam'
}

row_african_countries = {
    'DZ': 'Algeria', 'AO': 'Angola', 'BJ': 'Benin', 'BW': 'Botswana', 'BF': 'Burkina Faso', 'BI': 'Burundi',
    'CM': 'Cameroon', 'CV': 'Cape Verde', 'CF': 'Central African Republic', 'TD': 'Chad', 'KM': 'Comoros',
    'CG': 'Congo', 'CD': 'Congo DRC', 'DJ': 'Djibouti', 'EG': 'Egypt', 'EH': 'Western Sahara', 'GQ': 'Equatorial Guinea', 'ER': 'Eritrea',
    'ET': 'Ethiopia', 'GA': 'Gabon', 'GM': 'Gambia, The', 'GH': 'Ghana', 'GN': 'Guinea', 'GW': 'Guinea-Bissau',
    'CI': 'Ivory Coast', 'KE': 'Kenya', 'LS': 'Lesotho', 'LR': 'Liberia', 'LY': 'Libya', 'MG': 'Madagascar',
    'MW': 'Malawi', 'ML': 'Mali', 'MR': 'Mauritania', 'MU': 'Mauritius', 'MA': 'Morocco', 'MZ': 'Mozambique',
    'NA': 'Namibia', 'NE': 'Niger', 'NG': 'Nigeria', 'RW': 'Rwanda', 'ST': 'São Tomé and Príncipe', 'SN': 'Senegal',
    'SL': 'Sierra Leone', 'SO': 'Somalia', 'SZ': 'Eswatini',
    'SD': 'Sudan', 'TZ': 'Tanzania', 'TG': 'Togo', 'TN': 'Tunisia', 'UG': 'Uganda', 'ZM': 'Zambia', 'ZW': 'Zimbabwe'
}

row_american_countries = {
    'AR': 'Argentina', 'BS': 'Bahamas, The', 'BZ': 'Belize', 'BO': 'Bolivia',
    'CL': 'Chile', 'CO': 'Colombia', 'CR': 'Costa Rica', 'CU': 'Cuba', 'DO': 'Dominican Republic', 'EC': 'Ecuador',
    'SV': 'El Salvador', 'GT': 'Guatemala', 'GY': 'Guyana', 'HT': 'Haiti', 'HN': 'Honduras', 'JM': 'Jamaica',
    'NI': 'Nicaragua', 'PA': 'Panama', 'PY': 'Paraguay', 'PE': 'Peru', 'PR': 'Puerto Rico',
    'SR': 'Suriname', 'TT': 'Trinidad and Tobago', 'UY': 'Uruguay', 'VE': 'Venezuela'
}

row_middle_eastern_countries = {
    'AF': 'Afghanistan', 'AM': 'Armenia',
    'IR': 'Iran', 'IQ': 'Iraq', 'IL': 'Israel', 'JO': 'Jordan', 'KW': 'Kuwait', 'LB': 'Lebanon', 'OM': 'Oman',
    'QA': 'Qatar', 'SA': 'Saudi Arabia', 'SY': 'Syria', 'AE': 'United Arab Emirates',
    'YE': 'Yemen'
}

exio_regions = exio3_11.get_regions()
row_regions = {
    "WA": "Asia and pacific",
    "WE": "Europe",
    "WF": "Africa",
    "WM": "Middle east",
    "WL": "America"
}

def get_r_for_row_region(row_region):
    if row_region == "WA":
       return len(row_asia_pacific_countries) 
    elif row_region == "WE":
        return len(row_eu_countries)
    elif row_region == "WF":
        return len(row_african_countries)
    elif row_region == "WM":
        return len(row_middle_eastern_countries)
    elif row_region == "WL":
        return len(row_american_countries)

df_r = pd.DataFrame(index=exio_regions, columns=["R"])
# fill R vales: if regions key is not in row_regions (direct matches), set R to 1
# otherwise set R to the number of countries in the region which represents how the impact is assumed to be distributed
for region in df_r.index:
    if region in row_regions:
        df_r.loc[region, "R"] = get_r_for_row_region(region)
    else:
        df_r.loc[region, "R"] = 1

In [88]:
# harmonize regions between lc-impact and exiobase
all_row_region_keys = list(row_eu_countries.keys()) + list(row_asia_pacific_countries.keys()) + list(row_african_countries.keys()) + list(row_american_countries.keys()) + list(row_middle_eastern_countries.keys())
exio_regions_without_row = df_r.index[~df_r.index.isin(row_regions.keys())]
all_regions = list(all_row_region_keys) + list(exio_regions_without_row)
print(len(all_regions))

# check uniqueness of the regions
unique_regions = set(all_regions)
print(len(unique_regions))

# find out non unique regions
non_unique_regions = [region for region in unique_regions if all_regions.count(region) > 1]
print("Non unique regions: ", non_unique_regions)

# find out which regions are not in the lc-impact data
missing_regions = [region for region in unique_regions if region not in lci_water["Country_Code"].unique()]
print("Missing regions: ", missing_regions)

# find out regions that are in lci-water but not in the all_regions
lci_water_regions = lci_water["Country_Code"].unique()
missing_from_all_regions = [region for region in lci_water_regions if region not in unique_regions]
print("Missing from all regions: ", missing_from_all_regions)

# Malta is missing from the lc-impact data, use 9.02E-15 (continental average for europe)

185
185
Non unique regions:  []
Missing regions:  ['MT']
Missing from all regions:  [None]


In [100]:
dr_s.loc["WE"]

,"(AT, Paddy rice)","(AT, Wheat)","(AT, Cereal grains nec)","(AT, Vegetables, fruit, nuts)","(AT, Oil seeds)","(AT, Sugar cane, sugar beet)","(AT, Plant-based fibers)","(AT, Crops nec)","(AT, Cattle)","(AT, Pigs)",...,"(WM, Paper for treatment: landfill)","(WM, Plastic waste for treatment: landfill)","(WM, Inert/metal/hazardous waste for treatment: landfill)","(WM, Textiles waste for treatment: landfill)","(WM, Wood waste for treatment: landfill)","(WM, Membership organisation services n.e.c. (91))","(WM, Recreational, cultural and sporting services (92))","(WM, Other services (93))","(WM, Private households with employed persons (95))","(WM, Extra-territorial organizations and bodies)"
sector,,,,,,,,,,,,,,,,,,,,,
Paddy rice,8.381848e-08,0.000009,0.000037,0.000004,0.000022,0.000003,0.000020,0.000004,0.000025,0.000138,...,0.000006,0.000005,0.000006,0.000007,0.000006,0.000010,0.000010,0.000012,0.000007,NaN
Wheat,6.731463e-07,0.001028,0.000419,0.000018,0.000081,0.000013,0.000133,0.000049,0.000117,0.000354,...,0.000482,0.000482,0.000476,0.000484,0.000461,0.001610,0.000765,0.001080,0.000456,NaN
Cereal grains nec,3.385886e-06,0.000230,0.129307,0.000081,0.000390,0.000065,0.000899,0.000156,0.000944,0.002541,...,0.004902,0.005104,0.004843,0.005213,0.004971,0.004344,0.006961,0.009380,0.004576,NaN
"Vegetables, fruit, nuts",2.073370e-07,0.000021,0.000072,0.003866,0.000048,0.000007,0.000043,0.000008,0.000071,0.000316,...,0.000054,0.000049,0.000054,0.000039,0.000044,0.000053,0.000044,0.000072,0.000035,NaN
Oil seeds,2.673869e-07,0.000021,0.000052,0.000008,0.013067,0.000006,0.000046,0.000008,0.000077,0.000258,...,0.000160,0.000131,0.000161,0.000071,0.000071,0.000139,0.000114,0.000207,0.000112,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Membership organisation services n.e.c. (91),0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
"Recreational, cultural and sporting services (92)",0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
Other services (93),0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN


In [ ]:
# build dr_u from dr_s
dr_u = dr_s.copy()
# drop rest of the world regions
#dr_u.drop(index=row_regions.keys(), inplace=True)

exio_countries = dr_u.index.get_level_values(0).unique()
not_in_exio = [region for region in all_regions if region not in exio_countries]
print("Not in exio: ", len(not_in_exio))

# Add missing regions to the dr_u dataframe
for region in not_in_exio:
    if region in row_eu_countries:
        values_to_add = dr_s.loc["WE"].copy()
    elif region in row_asia_pacific_countries:
        values_to_add = dr_s.loc["WA"].copy()
    elif region in row_african_countries:
        values_to_add = dr_s.loc["WF"].copy()
    elif region in row_american_countries:
        values_to_add = dr_s.loc["WL"].copy()
    elif region in row_middle_eastern_countries:
        values_to_add = dr_s.loc["WM"].copy()
    else:
        raise Exception("Region not found in any of the row regions: ", region)
    
    # Add the new row with the correct MultiIndex
    for sub_index in values_to_add.index.unique():
        dr_u.loc[(region, sub_index)] = values_to_add

Not in exio:  141


KeyError: 'Paddy rice'

In [ ]:
# use 2019 impact factors for calculating dr_f
# dr_f represents the impact factors for each impact region i driven by consumption in region j product sector k

dr_f = df_s.copy()
exio3_19.satellite_agg.M.loc["Water Consumption Blue -– Total"].loc[("AT","Wheat")] # TODO: is this needed?
blue_water_total = exio3_19.satellite_agg.M.loc["Water Consumption Blue -– Total"]
scalars = {} # multipliers for each column

# multiply every entry of dr_s by the respective column value from exio3_satellite_agg.M
for col_name,col in df_s.items():
    scalars[col_name] = blue_water_total[col_name]

dr_f = dr_f * scalars

In [ ]:
# apply impact factors from lc-impact to every impact region i to get PDF for 1€ consumption in region j product k

# sum up the columns to get PDF/€ factors for all consumption regions and products